## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-05-01-08-51 +0000,nyt,Trump Is Renaming the Defense Department as th...,https://www.nytimes.com/2025/09/04/us/politics...
1,2025-09-05-01-08-34 +0000,bbc,Trump signs order to cut tariffs on Japanese c...,https://www.bbc.com/news/articles/cr70rg7ejdpo...
2,2025-09-05-01-03-30 +0000,nyt,Biden Has Surgery to Remove Skin Cancer,https://www.nytimes.com/2025/09/04/us/politics...
3,2025-09-05-01-02-58 +0000,missionlocal,"ICE arrests five asylum-seekers in S.F., despi...",https://missionlocal.org/2025/09/ice-arrest-fe...
4,2025-09-05-01-00-00 +0000,wsj,A renegade Republican is escalating his war wi...,https://www.wsj.com/politics/policy/trump-mass...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2352/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,73
238,court,16
39,out,14
7,tariffs,13
419,jr,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
223,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...,166
5,2025-09-05-01-00-00 +0000,wsj,President Trump’s sprint to take control of th...,https://www.wsj.com/politics/policy/trump-fed-...,149
28,2025-09-04-23-32-00 +0000,wsj,President Trump was within his authority when ...,https://www.wsj.com/politics/national-security...,143
18,2025-09-05-00-19-00 +0000,wsj,President Trump is expected to sign an executi...,https://www.wsj.com/politics/national-security...,134
254,2025-09-04-15-00-00 +0000,wsj,The BLS will publish its first jobs report Fri...,https://www.wsj.com/economy/jobs/jobs-report-a...,133


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
223,166,2025-09-04-16-20-00 +0000,wsj,President Trump’s campaign to bring the Federa...,https://www.wsj.com/economy/central-banking/tr...
107,70,2025-09-04-21-22-00 +0000,wsj,The Florida immigration detention facility kno...,https://www.wsj.com/politics/policy/florida-al...
105,64,2025-09-04-21-25-00 +0000,wsj,Health Secretary Robert F. Kennedy Jr. was gri...,https://www.wsj.com/politics/policy/rfk-jr-cdc...
212,64,2025-09-04-16-36-57 +0000,nypost,CBS News staffers ‘freaking out’ as new Paramo...,https://nypost.com/2025/09/04/media/paramount-...
169,55,2025-09-04-18-50-00 +0000,wsj,The Justice Department opened a criminal inves...,https://www.wsj.com/politics/policy/lisa-cook-...
84,48,2025-09-04-21-54-34 +0000,nyt,"After Boat Strike, Rubio Says U.S. Will Help O...",https://www.nytimes.com/2025/09/04/world/ameri...
116,46,2025-09-04-21-07-20 +0000,nyt,Portugal Searches for Cause of Lisbon Funicula...,https://www.nytimes.com/2025/09/04/world/europ...
368,44,2025-09-04-02-45-04 +0000,cbc,Trump administration takes tariffs fight to U....,https://www.cbc.ca/news/world/trump-tariffs-ap...
266,42,2025-09-04-14-00-00 +0000,wsj,Benjamin Franklin arrived in France in 1776 to...,https://www.wsj.com/politics/policy/ben-frankl...
300,37,2025-09-04-11-22-18 +0000,nypost,Kim Jong Un vows to ‘fully support’ Russia’s a...,https://nypost.com/2025/09/04/world-news/north...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
